In [1]:
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models.resnet import ResNet, BasicBlock
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from torch.utils.data import DataLoader,Dataset
import torchvision.utils
import numpy as np
import random
from PIL import Image
import PIL.ImageOps
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.autonotebook import tqdm as tqdm_jupyter
from tqdm import tqdm as tqdm_bash
import inspect
import os
import time
from siamese_tools import *

C:\Users\LionHeart\Anaconda2\envs\Project Sidewalk\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
pbar_type = "jupyter"#"bash"
epochs = 20
batch_size = 64

model_name = "skw.base_cnn.v1"
model_path = os.path.join("models",model_name+".ep_"+str(epochs))
if not os.path.exists(model_path):
    os.makedirs(model_path)

root_dir = "data"
train_dir = os.path.join(root_dir,"train")
test_dir = os.path.join(root_dir,"test")

im_train_dset = dset.ImageFolder(root=train_dir)
im_test_dset = dset.ImageFolder(root=test_dir)

## Basic CNN Transformations
#transformation = transforms.Compose([transforms.Resize((100,100)),
#                                     transforms.ToTensor()])

## ResNet Transformations
transformation = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

train_siamese_dset = SiameseDataset(imageFolderDataset=im_train_dset, transform=transformation)
val_siamese_dset = SiameseDataset(imageFolderDataset=im_test_dset, transform=transformation)
train_loader = DataLoader(train_siamese_dset,shuffle=True,num_workers=0,batch_size=batch_size)
val_loader = DataLoader(val_siamese_dset,shuffle=True,num_workers=0,batch_size=batch_size)

loss_function = ContrastiveLoss()

print("CUDA availability:",torch.cuda.is_available())

CUDA availability: True


In [3]:
start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

losses = []
batches = len(train_loader)
val_batches = len(val_loader)
best_loss = 0

#model = SiameseNetwork().cuda() ## Basic CNN Network
model = SiameseResNet().cuda() ## ResNet Network
optimizer = optim.Adadelta(model.parameters())

for epoch in range(epochs):
    total_loss = 0
    
    progress = None
    
    if pbar_type == "bash":
        progress = tqdm_bash(enumerate(train_loader,0), total=batches, unit=" batches", desc="Loss: ", position=0, leave=True)
    if pbar_type == "jupyter":
        progress = tqdm_jupyter(enumerate(train_loader,0), total=batches, unit=" batches", desc="Loss: ")
    
    model.train()
    
    for i,data in progress:
        img0, img1 , label = data
        img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
        optimizer.zero_grad()
        output1, output2 = model(img0,img1)
        loss = loss_function(output1,output2,label)
        loss.backward()
        optimizer.step()

        current_loss = loss.item()
        total_loss += current_loss

        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    curr_loss = total_loss/batches
    print(f"Epoch {epoch+1}/{epochs}, training loss: {curr_loss}")
          
    torch.save(model.state_dict(),os.path.join(model_path,"epoch_"+str(epoch)+".pt"))
    if (curr_loss < best_loss) or (best_loss == 0):
        best_loss = curr_loss
        print("Saving new best model at epoch {}".format(epoch))
        torch.save(model.state_dict(),os.path.join(model_path,"best.pt"))
        
print(f"Training time: {time.time()-start_ts}s")

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 11.00 GiB total capacity; 2.70 GiB already allocated; 0 bytes free; 50.39 MiB cached)